# Example of using SpecialtyInsurance Simulator


In [1]:
### 1. Set Simulation Parameters

import os
from logger.arguments import get_arguments

# Read arguments from logger.arguments
sim_args, manager_args, broker_args, syndicate_args, reinsurancefirm_args, shareholder_args, risk_args = get_arguments()

# Reset arguments
sim_args["max_time"] = 300   # Simulation time span unit day
manager_args["lead_top_k"] = 3   # Number of syndicates competing for the lead quote
manager_args["follow_top_k"] = 2   # Number of syndicates following the lead strategy
broker_args["num_brokers"] = 1   # Number of brokers in the insurance market
syndicate_args["num_syndicates"] = 3   # Number of syndicates in the insurance market
shareholder_args["num_shareholders"] = 1   # Number of shareholders in the insurance market
risk_args["num_risks"] = 1  # Number of risks
risk_args["num_categories"] = 4  # Number of risk categories

# No reinsurance mechanism included in this stage
with_reinsurance = False   

# Nomber of risk models loaded to all syndicates
num_risk_models = 1   


In [2]:
### 2. Generate Catastrophes

from environment.risk_generator import RiskGenerator

# Create catastrophe list and catastrophe configurations
catastrophes, risk_model_configs = RiskGenerator(num_risk_models, sim_args, risk_args).generate_risks()
print("catastrophe starts at time", catastrophes[0].get("risk_start_time"))


catastrophe starts at time 134


In [3]:
### 3. Generate Insurance Market

from environment.market_generator import MarketGenerator

# Create lists of brokers, syndicates, reinsurancefirms, and shareholders
brokers, syndicates, reinsurancefirms, shareholders = MarketGenerator(with_reinsurance, 
                                                                      num_risk_models, 
                                                                      sim_args, 
                                                                      broker_args, 
                                                                      syndicate_args, 
                                                                      reinsurancefirm_args, 
                                                                      shareholder_args, 
                                                                      risk_model_configs).generate_agents()
for broker_id in range(len(brokers)):
    print(brokers[broker_id].data())
for syndicate_id in range(len(syndicates)):
    print(syndicates[syndicate_id].data())
    

{'broker_id': '0', 'broker_risk': [], 'broker_quote': []}
{'syndicate_id': '0', 'initial_capital': 10000000, 'current_capital': 10000000, 'premium_internal_weight': 0.5, 'interest_rate': 0.001, 'play_leader_in_contracts': [], 'play_follower_in_contracts': [], 'loss_experiency_weight': 0.2, 'volatility_weight': 0, 'underwriter_markup_recency_weight': 0.2, 'upper_premium_limit': 1.2, 'lower_premium_limit': 0.85, 'premium_reserve_ratio': 0.5, 'minimum_capital_reserve_ratio': 1, 'maximum_scaling_factor': 1, 'market_entry_probability': 0.3, 'exit_capital_threshold': 0.6, 'exit_time_limit': 24, 'premium_sensitivity': 5, 'acceptance_threshold_friction': 0.9}
{'syndicate_id': '1', 'initial_capital': 10000000, 'current_capital': 10000000, 'premium_internal_weight': 0.5, 'interest_rate': 0.001, 'play_leader_in_contracts': [], 'play_follower_in_contracts': [], 'loss_experiency_weight': 0.2, 'volatility_weight': 0, 'underwriter_markup_recency_weight': 0.2, 'upper_premium_limit': 1.2, 'lower_premiu

In [4]:
### 4. Input risk from broker

from environment.event_generator import EventGenerator

current_time = 0
broker_risk_events = EventGenerator(risk_model_configs).generate_risk_events(sim_args, brokers, catastrophes)

for i in range(len(broker_risk_events)):
    print("risk_id:", broker_risk_events[i].risk_id, "broker_id:", broker_risk_events[i].broker_id, "risk_start_time:", broker_risk_events[i].risk_start_time,
         "risk_end_time:", broker_risk_events[i].risk_end_time, "risk_factor:", broker_risk_events[i].risk_factor,
         "risk_category:", broker_risk_events[i].risk_category, "risk_value:", broker_risk_events[i].risk_value)


risk_id: 0 broker_id: 0 risk_start_time: 0 risk_end_time: 360 risk_factor: 0.5343209816688624 risk_category: 0 risk_value: 1000.0
risk_id: 1 broker_id: 0 risk_start_time: 1 risk_end_time: 361 risk_factor: 0.5343209816688624 risk_category: 0 risk_value: 1000.0
risk_id: 2 broker_id: 0 risk_start_time: 2 risk_end_time: 362 risk_factor: 0.5343209816688624 risk_category: 0 risk_value: 1000.0
risk_id: 3 broker_id: 0 risk_start_time: 3 risk_end_time: 363 risk_factor: 0.5343209816688624 risk_category: 0 risk_value: 1000.0
risk_id: 4 broker_id: 0 risk_start_time: 4 risk_end_time: 364 risk_factor: 0.5343209816688624 risk_category: 0 risk_value: 1000.0
risk_id: 5 broker_id: 0 risk_start_time: 5 risk_end_time: 365 risk_factor: 0.5343209816688624 risk_category: 0 risk_value: 1000.0
risk_id: 6 broker_id: 0 risk_start_time: 6 risk_end_time: 366 risk_factor: 0.5343209816688624 risk_category: 0 risk_value: 1000.0
risk_id: 7 broker_id: 0 risk_start_time: 7 risk_end_time: 367 risk_factor: 0.5343209816688

In [5]:
### 5. Create Multi-agent Environment to get access to the market performance

import gym
from environment.environment import SpecialtyInsuranceMarketEnv
from environment.event_generator import EventGenerator
from manager.ai_model.action import Action
from manager import *

class MultiAgentBasedModel(SpecialtyInsuranceMarketEnv):

    def __init__(self, sim_args, manager_args, brokers, syndicates, reinsurancefirms, shareholders, catastrophes, risk_model_configs, with_reinsurance, num_risk_models, dt = 1):

        self.sim_args = sim_args
        self.maxstep = self.sim_args["max_time"]
        self.manager_args = manager_args
        self.brokers = brokers
        self.initial_brokers = brokers
        self.syndicates = syndicates
        self.initial_syndicates = syndicates
        self.reinsurancefirms = reinsurancefirms
        self.initial_reinsurancefirms = reinsurancefirms
        self.shareholders = shareholders
        self.initial_shareholders = shareholders
        self.risks = catastrophes
        self.initial_risks = catastrophes
        self.risk_model_configs = risk_model_configs
        self.with_reinsurance = with_reinsurance
        self.num_risk_models = num_risk_models
        self.dt = dt
        self.mm = None
        self.event_handler = None

        # Active syndicate list
        self.syndicate_active_list = []
        # Initialise events, actions, and states 
        self.broker_risk_events = []
        self.action_map_dict = {}
        self.state_encoder_dict = {}

        # Define Action Space, Define Observation Space
        self.n = len(self.syndicates)
        self.agents = {self.syndicates[i].syndicate_id for i in range(self.n)}
        self._agent_ids = set(self.agents)
        self.dones = set()
        self._spaces_in_preferred_format = True
        self.observation_space = gym.spaces.Dict({self.syndicates[i].syndicate_id: self.obs_space_creator() for i in range(self.n)})
        self.action_space = gym.spaces.Dict({self.syndicates[i].syndicate_id: self.set_action_space() for i in range(self.n)})

        super(MultiAgentBasedModel, self).__init__(sim_args = self.sim_args, 
                                                   manager_args = self.manager_args , 
                                                   brokers = self.brokers, 
                                                   syndicates = self.syndicates, 
                                                   reinsurancefirms = self.reinsurancefirms, 
                                                   shareholders = self.shareholders, 
                                                   risks = self.risks, 
                                                   risk_model_configs = self.risk_model_configs, 
                                                   with_reinsurance = self.with_reinsurance, 
                                                   num_risk_models = self.num_risk_models,
                                                   dt = 1)
        # Reset the environmnet
        self.reset()

    def reset(self, seed = None, options = None):
        super().reset(seed = seed)
        
        # Reset the environment to an initial state
        self.brokers = self.initial_brokers
        self.syndicates = self.initial_syndicates
        self.reinsurancefirms = self.initial_reinsurancefirms
        self.shareholders = self.initial_shareholders
        self.risks = self.initial_risks
        # Broker risk event daily: broker generate risk according to poisson distribution
        self.broker_risk_events = broker_risk_events
        self.event_handler = EventHandler(self.maxstep, self.broker_risk_events)
        # Initiate market manager
        self.mm = MarketManager(self.maxstep, self.manager_args, self.brokers, self.syndicates, self.reinsurancefirms, self.shareholders, self.risks, self.risk_model_configs, self.with_reinsurance, self.num_risk_models, self.broker_risk_events, self.event_handler)
        self.mm.evolve(self.dt)
        
        # Set per syndicate active status and build status list
        self.syndicate_active_list = []   # Store syndicates currently in the market
        for syndicate_id in self.mm.market.syndicates:
            if self.mm.market.syndicates[syndicate_id].status == True:
                self.syndicate_active_list.append(syndicate_id)

        # Create action map and state list
        info_dict = {}
        for syndicate_id in self.syndicate_active_list:
            self.action_map_dict[syndicate_id] = self.action_map_creator(self.mm.market.syndicates[syndicate_id], 0)
            #self.state_encoder_dict[syndicate_id] = self.state_encoder(syndicate_id)
            self.state_encoder_dict = {i: self.observation_space[i].sample() for i in self.agents}
            info_dict[syndicate_id] = self._get_info()

        # Initiate time step
        self.timestep = -1
        self.step_track = 0
        self.log = []

        return self.state_encoder_dict, info_dict

    def compute_reward(self, action, syndicate_id):

        market = self.mm.market
        # calculate reward function
        r = [0.0] * 4

        # For each insurable risk being accepted +1 or refused -1
        if(self.timestep <= self.maxstep):
            for broker_id in range(len(market.brokers)):
                for risk in range(len(market.brokers[broker_id].risks)):
                    for contract in range(len(market.brokers[broker_id].underwritten_contracts)):
                        if market.brokers[broker_id].risks[risk]["risk_id"] == market.brokers[broker_id].underwritten_contracts[contract]["risk_id"]:
                            r[0] += 1
                        else:
                            r[0] -= 1

        # For each claim being paied +1 or refused -1
        if(self.timestep <= self.maxstep):
            for claim in range(len(market.syndicate[syndicate_id].paid_claim)):
                if market.syndicate[syndicate_id].paid_claim[claim]["status"] == True:
                    r[1] += 1
                else:
                    r[1] -= 1

        # Profit and Bankruptcy       
        if(self.timestep <= self.maxstep):
            initial_capital = self.syndicate_status[syndicate_id].initial_capital
            current_capital = self.syndicate_status[syndicate_id].update_capital()
            r[2] += current_capital - initial_capital
            if (current_capital - initial_capital) < 0:
                r[3] -= 10000

        # Sum reward
        reward = 0.0
        reward += np.sum(r)

        return reward     
    
    def state_encoder(self, syndicate_id):

        ### Observation Space:             
        obs = []
        market = self.mm.market
        for risk in range(len(market.risks)):
            if market.risks[risk]["risk_start_time"] == self.timestep:
                # Catastrophe risk category and risk value
                obs.append(market.risks[risk]["risk_category"])
                obs.append(market.risks[risk]["risk_value"])

        # Syndicates status current capital in 
        obs.append(self.syndicates[syndicate_id]["current_capital"])
        for num in range(len(self.syndicates[syndicate_id].current_capital_category)):
            obs.append(self.syndicates[syndicate_id].current_capital_category[num])
            
        return obs

    def set_action_space(self):
        return gym.spaces.Box(0.0, 0.9, dtype = np.float32)
   

In [6]:
### 6. Register environment and train the model

import numpy as np
from ray.tune.registry import register_env
from ray import tune
from ray.rllib.algorithms.ppo import PPO
from ipywidgets import IntProgress
from ray.rllib.policy.policy import PolicySpec
from gymnasium.spaces import Box

insurance_args = {"sim_args": sim_args,
    "manager_args": manager_args,
    "brokers": brokers,
    "syndicates": syndicates,
    "reinsurancefirms": reinsurancefirms,
    "shareholders": shareholders,
    "catastrophes": catastrophes,
    "risk_model_configs": risk_model_configs,
    "with_reinsurance": with_reinsurance,
    "num_risk_models": num_risk_models}

def env_creator(env_config):
    return MultiAgentBasedModel(**env_config)

def ppo_trainer_creator(insurance_args):
    low, high = [], []
    n = len(syndicates)
    low.extend([0.0, 0.0])
    high.extend([10.0, 10000000.0]) # Number of risk category, risk limit, current capital
    for num in range(risk_model_configs[0]["num_categories"]):
        low.append(-10000000.0)
        high.append(30000000.0)
    config={"log_level": "ERROR",
            "env": "SpecialtyInsuranceMarket-validation",
            "num_workers": 1,
            "framework": "tf",
            "model": {
                "fcnet_hiddens": [32, 16],
                "fcnet_activation": "relu",
                },
            "evaluation_interval": 2, #num of training iter between evaluation
            "evaluation_duration": 20,
            "num_gpus": 0,
            "multiagent": {
            "policies": {
                syndicates[i].syndicate_id: PolicySpec(observation_space=Box(np.array(low, dtype=np.float32), np.array(high, dtype=np.float32)), action_space=Box(0.5, 0.9, dtype = np.float32)) for i in range(n)
                
            },
            "policies_to_train": ["0"]
            },
            "env_config": insurance_args}
    trainer = PPO(config=config)
    return trainer

# Folder for recording
top_dir = "noreinsurance_" + "_model_" + str(num_risk_models)

# Register environment
register_env("SpecialtyInsuranceMarket-validation", env_creator)

# The number of training iteration for the RL agent
num_training = 10

trainer = ppo_trainer_creator(insurance_args)
# Number of training iterations
num_training = 30
for n in range(num_training):
    # Create a path to store the trained agent for each iteration
    model_filepath = f"{top_dir}/{str(n)}/saved_models"
        
    num_episode = 10

    # A training iteration includes parallel sample collection by the environment workers 
    # as well as loss calculation on the collected batch and a model update.

    bar = IntProgress(min=0, max=num_episode)
    display(bar)
    list_mean_rewards = []
    list_min_rewards = []
    list_max_rewards = []
    list_train_step = []

    for i in range(num_episode):
        trainer.train()        
        print("Progress:", i+1, "/", num_episode, end="\r")
        bar.value += 1
        if (i+1) % config["evaluation_interval"] == 0:
            list_mean_rewards.append(trainer.evaluation_metrics["evaluation"]["episode_reward_mean"])
            list_min_rewards.append(trainer.evaluation_metrics["evaluation"]["episode_reward_min"])
            list_max_rewards.append(trainer.evaluation_metrics["evaluation"]["episode_reward_max"])
            list_train_step.append(i+1)


/Users/k2472543/Library/Python/3.8/lib/python/site-packages/ray/rllib/algorithms/algorithm.py:483: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
`UnifiedLogger` will be removed in Ray 2.7.
  return UnifiedLogger(config, logdir, loggers=None)
/Users/k2472543/Library/Python/3.8/lib/python/site-packages/ray/tune/logger/unified.py:53: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
The `JsonLogger interface is deprecated in favor of the `ray.tune.json.JsonLoggerCallback` interface and will be removed in Ray 2.7.
  self._loggers.append(cls(self.config, self.logdir, self.trial))
/Users/k2472543/Library/Python/3.8/lib/python/site-packages/ray/tune/logger/unified.py:53: RayDeprecationWarning: This API is deprecated

IndexError: list index out of range

In [ ]:
### 7. Test the trained model performance

def trainer_restore(self, top_dir, n):
    if n <= 9:
        path0 = top_dir
        path1 = str(n-1)
        path2 = "saved_models"
        path3 = "checkpoint_"+str(0)+str(0)+str(0)+str(0)+str(0)+str(n)
        path4 = "rllib_checkpoint.json"
    elif 9 < n <= 99:
        path0 = top_dir
        path1 = str(n-1)
        path2 = "saved_models"
        path3 = "checkpoint_"+str(0)+str(0)+str(0)+str(0)+str(n)
        path4 = "rllib_checkpoint.json"
    elif 99 < n <= 999:
        path0 = top_dir
        path1 = str(n-1)
        path2 = "saved_models"
        path3 = "checkpoint_"+str(0)+str(0)+str(0)+str(n)
        path4 = "rllib_checkpoint.json"

    # Join various path components
    self.trainer.restore(os.path.join(path0, path1, path2, path3, path4))

insurance_args = {"sim_args": self.sim_args,
        "manager_args": self.manager_args,
        "brokers": self.brokers,
        "syndicates": self.syndicates,
        "reinsurancefirms": self.reinsurancefirms,
        "shareholders": self.shareholders,
        "risks": self.risks,
        "risk_model_configs": self.risk_model_configs,
         "with_reinsurance": self.with_reinsurance,
        "num_risk_models": self.num_risk_models}

validation_episodes = 1
        
for epi in range(validation_episodes):
    env = SpecialtyInsuranceMarket(**insurance_args)
    
    print(f"\nepisode: {epi} | ")
    total_steps = 0
    done = {"__all__": False}
    all_rewards[epi] = {}
    
    obs = env.reset()
    
    while not done["__all__"]:
        if total_steps % 20 == 0: print(".", end="")
        
        action_dict = trainer.compute_actions(obs)  
        total_steps += 1
        
        obs, reward, done, info = env.step(action_dict, 
                                          draw_to_file=True)
        for k, v in reward.items():
            if k not in all_rewards[epi]:
                all_rewards[epi][k] = [v]
            else:
                all_rewards[epi][k].append(v)

        return all_rewards


In [ ]:
### Main function run the simulation, two syndicates will be chosen to compete for the leader position
from manager.ai_model.runner import AIRunner
from manager.game_model.runner import GameRunner

model = 0
if model == 0: 
    runner = AIRunner(sim_args, manager_args, brokers, syndicates, reinsurancefirms, shareholders, catastrophes, risk_model_configs, with_reinsurance, num_risk_models)
elif model == 1:
    runner = GameRunner(sim_args, manager_args, brokers, syndicates, reinsurancefirms, shareholders, catastrophes, risk_model_configs, with_reinsurance, num_risk_models)
runner.run()


In [ ]:
from environment.event.add_catastrophe import AddCatastropheEvent
from environment.event.add_attritionalloss import AddAttritionalLossEvent
from environment.event.add_risk import AddRiskEvent
from environment.event.add_premium import AddPremiumEvent
from environment.event.add_claim import AddClaimEvent
from environment.event_generator import EventGenerator
from manager.event_handler import EventHandler
from environment.market import NoReinsurance_RiskOne
import numpy as np

catastrophe_events = EventGenerator(risk_model_configs).generate_catastrophe_events(catastrophes)
attritional_loss_events = EventGenerator(risk_model_configs).generate_attritional_loss_events(sim_args, catastrophes)
broker_risk_events = EventGenerator(risk_model_configs).generate_risk_events(brokers, catastrophes)
broker_premium_events = []
broker_claim_events = []
time = 0
market = NoReinsurance_RiskOne(time, sim_args["max_time"], manager_args, brokers, syndicates, 
                               shareholders, catastrophes, risk_model_configs, catastrophe_events, 
                               attritional_loss_events, broker_risk_events, broker_premium_events, 
                               broker_claim_events)
event_handler = EventHandler(sim_args["max_time"], catastrophe_events, attritional_loss_events, broker_risk_events, broker_premium_events, broker_claim_events)

step_time = 1
market_start_time = market.time
market_end_time = market.time + step_time

upcoming_catastrophe = [
            e.risk_id for e in event_handler.upcoming_catastrophe.values() if isinstance(e, AddCatastropheEvent)
        ]
upcoming_attritional_loss = [
            e.risk_id for e in event_handler.upcoming_attritional_loss.values() if isinstance(e, AddAttritionalLossEvent)
        ]
upcoming_broker_risk = [
            e.risk_id for e in event_handler.upcoming_broker_risk.values() if isinstance(e, AddRiskEvent)
        ]
upcoming_broker_premium = [
            e.risk_id for e in event_handler.upcoming_broker_premium.values() if isinstance(e, AddPremiumEvent)
        ]
upcoming_broker_claim = [
            e.risk_id for e in event_handler.upcoming_broker_claim.values() if isinstance(e, AddClaimEvent)
        ]
step_time = 1
market = event_handler.forward(market, step_time)

newly_added_catastrophe_events = {
            e.risk_id: e.risk_start_time
            for e in event_handler.completed_catastrophe.values()
            if isinstance(e, AddCatastropheEvent) and (e.risk_id in upcoming_catastrophe)
        }

newly_added_attritional_loss_events = {
            e.risk_id: e.risk_start_time
            for e in event_handler.completed_attritional_loss.values()
            if isinstance(e, AddAttritionalLossEvent) and (e.risk_id in upcoming_attritional_loss)
        }

newly_added_broker_risk_events = {
            e.risk_id: e.risk_start_time
            for e in event_handler.completed_broker_risk.values()
            if isinstance(e, AddRiskEvent) and (e.risk_id in upcoming_broker_risk)
        }

newly_added_broker_premium_events = {
            e.risk_id: e.risk_start_time
            for e in event_handler.completed_broker_premium.values()
            if isinstance(e, AddPremiumEvent) and (e.risk_id in upcoming_broker_premium)
        }

newly_added_broker_claim_events = {
            e.risk_id: e.risk_start_time
            for e in event_handler.completed_broker_claim.values()
            if isinstance(e, AddClaimEvent) and (e.risk_id in upcoming_broker_claim)
        }

catastrophe_event_start_times = np.array(
            [
                newly_added_catastrophe_events.get(risk_id)
                for risk_id in upcoming_catastrophe
                if newly_added_catastrophe_events.get(risk_id) != None
            ]
        )

attritional_loss_event_start_times = np.array(
            [
                newly_added_attritional_loss_events.get(risk_id)
                for risk_id in upcoming_attritional_loss
                if newly_added_attritional_loss_events.get(risk_id) != None
            ]
        )

broker_risk_event_start_times = np.array(
            [
                newly_added_broker_risk_events.get(risk_id)
                for risk_id in upcoming_broker_risk
                if newly_added_broker_risk_events.get(risk_id) != None
            ]
        )

broker_premium_event_start_times = np.array(
            [
                newly_added_broker_premium_events.get(risk_id)
                for risk_id in upcoming_broker_premium
                if newly_added_broker_premium_events.get(risk_id) != None
            ]
        )

broker_claim_event_start_times = np.array(
            [
                newly_added_broker_claim_events.get(risk_id)
                for risk_id in upcoming_broker_claim
                if newly_added_broker_claim_events.get(risk_id) != None
            ]
        )

event_start_times = np.concatenate((catastrophe_event_start_times,
                                    attritional_loss_event_start_times,
                                    broker_risk_event_start_times,
                                    broker_premium_event_start_times,
                                    broker_claim_event_start_times))

sorted_unique_start_times = np.sort(np.unique(event_start_times))

print(event_handler.completed_attritional_loss.values())